# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [4]:
import os
from FACET.Facet import Facet
from loguru import logger
import sys

os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

eegDataSet_filepath = "NiazyFMRI.edf"
motiondata_moosmann_filepath = "headmotiondata.tsv"
export_bids_path = "./bids_dir"

#configure logging
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("Facet.log", level="DEBUG")

#defining important parameters
window_size = 25 # defines the window size for averaging
upsampling_factor = 10 # defines the resampling factor
relative_trigger_position = -0.01 # defines the relative position of the artefact to the trigger.
relative_window_position = -0.5 # defines the relative position of the averaging window over the epochs.
moosmann_motion_threshold = 0.8 # defines the threshold for moosmann which determines if a motion data entry is significant or not
event_id_description_pairs={'trigger':1} # For exporting as bids. It defines the event id and its description.
regex_trigger_annotation_filter = r'\btrigger\b' # Annotations with the description 'trigger' are considered as triggers, declard for find triggers after reimporting from bids
regex_trigger_event_filter = r'\b1\b' # Events with the value 1 are considered as triggers, declared for edf stim channel
unwanted_bad_channels = ['EKG', 'EMG', 'EOG', 'ECG'] # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures=["SNR", "RMS", "RMS2", "MEDIAN"] # Measures that must be calculated and used for evaluation.

f = Facet()


## Second Step import EEG Data

In [5]:
f.import_EEG(eegDataSet_filepath, rel_trig_pos=relative_trigger_position, upsampling_factor=upsampling_factor, bads=unwanted_bad_channels)
f.find_triggers(regex_trigger_event_filter)
eeg = f.get_EEG()
f.export_as_bids(event_id_description_pairs)
f.import_from_bids(rel_trig_pos=relative_trigger_position, bads=unwanted_bad_channels)
eeg = f.get_EEG()

f.plot_EEG(title="after import")

2024-03-03 21:52:19.527 | INFO     | FACET.Facet:import_EEG:18 - Importing EEG from NiazyFMRI.edf
2024-03-03 21:52:19.733 | INFO     | FACET.Facet:find_triggers:28 - finding triggers
2024-03-03 21:52:22.471 | INFO     | FACET.Frameworks.Analytics:export_as_bids:71 - Exporting Channels: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'AF4', 'AF3', 'FC2', 'FC1', 'CP1', 'CP2', 'PO3', 'PO4', 'FC6', 'FC5', 'CP5', 'EMG', 'ECG', 'Status']


PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'bids_dir\\sub-subjectid\\ses-sessionid\\eeg\\sub-subjectid_ses-sessionid_task-corrected_eeg.edf'

## Third Step Preprocessing

In [ ]:
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(regex_trigger_annotation_filter) # Using Niazys data
f.get_analytics().print_analytics()
eeg = f.get_EEG()


## Fourth step apply AAS

In [ ]:
f.apply_AAS(window_size=window_size, rel_window_position=relative_window_position)

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )
eeg = f.get_EEG()



## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")

## Eighth Evaluate Results

In [ ]:
eeg_python = f.get_eeg()
f.add_to_evaluate(eeg_python, name="numpy_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies
logger.info(f.evaluate(measures=evaluation_measures))
f.get_correction().apply_ANC()#applying ANC
f.add_to_evaluate(f.get_eeg(), name="numpy_matrix_anc")
logger.info(f.evaluate(measures=evaluation_measures))
f.export_as_bids(event_id=event_id_description_pairs, bids_path=export_bids_path)

